In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
from numpy import exp, sqrt, log, pi

def heston_call_price(S, K, T, r, kappa, theta, sigma, rho, v0):
    """
    Price a European call option using the Heston model.

    Parameters:
    S (float): Initial stock price
    K (float): Strike price
    T (float): Time to maturity
    r (float): Risk-free rate
    kappa (float): Rate of mean reversion
    theta (float): Long-run average price of variance
    sigma (float): Volatility of volatility
    rho (float): Correlation between stock and variance
    v0 (float): Initial variance

    Returns:
    float: Price of the call option
    """
    i = complex(0, 1)

    def characteristic_function(phi):
        d = sqrt((rho * sigma * i * phi - kappa)**2 + (sigma**2) * (i * phi + phi**2))
        g = (kappa - rho * sigma * i * phi - d) / (kappa - rho * sigma * i * phi + d)
        C = r * i * phi * T + (kappa * theta / (sigma**2)) * (
            (kappa - rho * sigma * i * phi - d) * T - 2 * log((1 - g * exp(-d * T)) / (1 - g))
        )
        D = ((kappa - rho * sigma * i * phi - d) / sigma**2) * ((1 - exp(-d * T)) / (1 - g * exp(-d * T)))
        return exp(C + D * v0 + i * phi * log(S))

    def integrand(phi):
        integrand_value = (exp(-i * phi * log(K)) * characteristic_function(phi) / (i * phi)).real
        return integrand_value

    integral = quad(integrand, 0, np.inf)[0]
    call_price = S * 0.5 + (exp(-r * T) / pi) * integral - K * exp(-r * T) * 0.5
    return call_price

# Example parameters
S = 22475       # Initial stock price
K = 22500       # Strike price
T = 3/365         # Time to maturity
r = 0.065     # Risk-free rate
kappa = 0.05   # Rate of mean reversion
theta = 0.00  # Long-run average price of variance
sigma = 0.16  # Volatility of volatility
rho = -0.6   # Correlation between stock and variance
v0 = 0.01    # Initial variance

# Calculate the call price
price = heston_call_price(S, K, T, r, kappa, theta, sigma, rho, v0)
print("The price of the European call option is:", price)

The price of the European call option is: -6.509904011289109
